In [24]:
from maruti.imports.ml import *
maruti.imports.ml??
import numpy
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torch
import onnx
import torch
import numpy as np
import onnxruntime

Type:        module
String form: <module 'maruti.imports.ml' from '/home/akanksha/.local/lib/python3.8/site-packages/maruti/imports/ml.py'>
File:        ~/.local/lib/python3.8/site-packages/maruti/imports/ml.py
Source:     
from .general import *
from .general import __all__ as gen_all
import torch
import torch.nn as nn
import torchvision.transforms as torch_transforms
import torchvision
import maruti.torch as mtorch
import maruti.deepfake.dataset as mdata
import maruti
import maruti.deepfake as mfake
import numpy as np
import cv2
import maruti.vision as mvis
import pandas as pd
import torch.utils.data as tdata
import matplotlib.pyplot as plt
from torch.utils import data
import torch.optim as optim
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
__all__ = gen_all + ['mfake','mvis', 'cv2', 'mdata', 'tdata', 'pd', 'device', 'plt', 'np', 'torch', 'nn', 'torch_transforms',
                     'torchvision', 'mtorch', 'maruti', 'data', 'optim']


In [2]:
train_data_1 = pd.read_csv('~/Downloads/data.csv')

In [3]:
train_data_1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,52,1,4,160,331,0,0,94,1,2.5,?,?,?,1
290,54,0,3,130,294,0,1,100,1,0.0,2,?,?,1
291,56,1,4,155,342,1,0,150,1,3.0,2,?,?,1
292,58,0,2,180,393,0,0,110,1,1.0,2,?,7,1


In [4]:
train_data = np.array(train_data_1)

In [5]:
for j in range(13):
    count = 0
    add = 0
    for i in range(294):
        if train_data[i][j] != '?' and train_data[i][j] != None:
            count += 1
            if str(train_data[i][j]):
                add += int(train_data[i][j])
            else:
                add += train_data[i][j]
    mean = add/count
    for i in range(294):
        if train_data[i][j] == '?' or train_data[i][j] == None:
            train_data[i][j] = mean
train_data = train_data.astype(float)
count = 0
for i in range(294):
    for j in range(13):
        try:
            if int(train_data[i][j])>3:
                count += 1
        except:
            print(train_data[i][j])
            raise Exception
count

    

1588

In [6]:
temp = torch.tensor(train_data.astype(float))

In [7]:
std = temp.std(dim = 0)+1e-10
mean = temp.mean(dim = 0)+1e-10
std, mean

(tensor([7.8118e+00, 4.4753e-01, 9.6512e-01, 1.7596e+01, 6.4948e+01, 2.5196e-01,
         4.6008e-01, 2.3549e+01, 4.5988e-01, 9.0865e-01, 2.0099e-01, 1.0000e-10,
         4.9028e-01, 4.8098e-01], dtype=torch.float64),
 tensor([4.7827e+01, 7.2449e-01, 2.9830e+00, 1.3258e+02, 2.5085e+02, 6.9930e-02,
         2.1843e-01, 1.3913e+02, 3.0375e-01, 5.8605e-01, 1.8942e+00, 1.0000e-10,
         5.6429e+00, 3.6054e-01], dtype=torch.float64))

In [8]:
std.numpy().tolist(), mean.numpy().tolist()

([7.811812413249743,
  0.44753276424229776,
  0.9651167784527555,
  17.59646269974192,
  64.94793916841772,
  0.2519642873562598,
  0.4600804123315047,
  23.549459365962697,
  0.45987782297239505,
  0.9086479157256425,
  0.20099189593053107,
  1e-10,
  0.49027599477111933,
  0.4809770805966052],
 [47.8265306123449,
  0.7244897960183674,
  2.9829931973789114,
  132.58361774754027,
  250.8487084871849,
  0.06993007003006993,
  0.21843003422969284,
  139.1296928328645,
  0.30375426631160407,
  0.5860544218687075,
  1.8942307693307694,
  1e-10,
  5.642857142957142,
  0.36054421778707485])

In [9]:
train_data_1 = np.array(train_data)

In [10]:
class Dataset(tdata.Dataset):
    def __init__(self, train_data = train_data):
        self.train_data = train_data_1
    def __len__(self):
        return len(self.train_data)
    def __getitem__(self, i):
        data = np.array(self.train_data[i])
        x =  ((torch.tensor(data[:-1].astype(float),dtype=torch.float) - mean[:-1])/std[:-1]).float()
        y = data[-1]
        
        return x, torch.tensor(y, dtype=torch.float)
train_dataset, val_dataset = tdata.random_split(Dataset(),[200,94])

In [11]:
loader = tdata.DataLoader(train_dataset, shuffle=True,batch_size = 5)
val_loader = tdata.DataLoader(val_dataset, batch_size = 2)

In [12]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_linear = nn.Linear(13, 24)
        self.second_linear = nn.Linear(24, 32)
        self.third_linear = nn.Linear(32, 32)
        self.fourth_linear = nn.Linear(32, 8)
        self.fifth_linear = nn.Linear(8,1)
        self.bn = nn.BatchNorm1d(32)
        self.dropout = nn.Dropout(p = 0.4)
    def forward(self, batch):
#         print(batch)
        pred = self.first_linear(batch)
        pred = torch.relu(pred)
        
        pred = self.second_linear(pred)
        pred = torch.relu(pred)
        pred = self.dropout(pred)
        pred = self.third_linear(pred)
        pred = torch.relu(pred)
        pred = self.bn(pred)
        pred = self.fourth_linear(pred)
        pred = torch.relu(pred)
        
        pred = self.fifth_linear(pred)
        pred =  torch.sigmoid(pred.squeeze(1))
        return pred

In [13]:
model = Model()
learner = mtorch.Learner(model)

In [14]:
learner.compile(optim.Adam(model.parameters(),1e-3,weight_decay=1e-2), nn.BCELoss(),metrics = [mtorch.metrics.baccuracy,mtorch.metrics.bprecision,mtorch.metrics.brecall], callback = mtorch.callback.BoardLog())

In [15]:
learner.fit(50, loader, val_loader)

 Train Loss |  Val Loss  | baccuracy  | bprecision |  brecall   |    Time    |


0.65143126..|0.58755171..|0.73404252..|    1.0     |0.19354838..|   00:00    |


0.59710425..|0.53916800..|0.81914895..|0.85000002..|0.54838711..|   00:00    |


0.54688674..|0.48403197..|0.80851066..|0.80952382..|0.54838711..|   00:00    |


0.53183192..|0.42032915..|0.82978725..|0.82608693..|0.61290323..|   00:00    |


...

0.30070984..|0.38295018..|0.85106384..|0.77419352..|0.77419352..|   00:00    |


0.28595617..|0.39907401..|0.82978725..|0.75862067..|0.70967739..|   00:00    |


0.29349488..|0.38194668..|0.82978725..|0.77777779..|0.67741936..|   00:00    |


0.29448127..|0.39398384..|0.84042555..|0.78571426..|0.70967739..|   00:00    |


0.29482564..|0.37050685..|0.86170214..|0.80000001..|0.77419352..|   00:00    |
Total Time: 00:20, Best Score: 0.3300033211708069


In [16]:
model.first_linear.weight.data

tensor([[-1.0283e-01, -2.3629e-01,  2.4066e-02,  1.8537e-01,  1.5908e-01,
          8.0681e-02,  1.9038e-01,  1.9437e-02, -1.7763e-01, -9.6417e-02,
         -2.3670e-02,  1.1200e-02,  1.2798e-01],
        [-7.4004e-03, -9.9176e-03, -2.2967e-02,  8.0753e-02, -1.1226e-03,
         -3.7462e-02,  3.1255e-02,  1.8106e-01,  7.7656e-02, -2.6725e-02,
          1.4294e-02,  2.8187e-02,  8.1771e-02],
        [-8.8270e-02, -2.1300e-02, -8.1517e-05,  1.4567e-01, -2.5694e-02,
         -9.6213e-02,  2.2043e-01, -8.8678e-02,  3.2418e-02, -1.8853e-02,
          1.0554e-01,  9.2992e-02,  1.3265e-01],
        [ 1.3519e-01, -2.7924e-01,  2.2026e-01, -1.7962e-01,  8.3877e-02,
         -1.4311e-01,  2.0452e-01, -2.4755e-03,  5.2505e-02,  2.1885e-02,
         -1.5729e-01,  2.0147e-01, -9.4593e-02],
        [ 8.9236e-02, -1.9066e-01, -1.6342e-01, -1.2871e-01,  6.0511e-02,
         -1.7411e-01, -5.8356e-02,  7.9521e-02, -1.2518e-02,  7.8949e-02,
          1.0553e-01, -9.3540e-02, -3.4089e-04],
        [ 4.151

In [17]:
x = torch.randn(1, 13, requires_grad=True)

In [18]:
torch_out = model(x)

In [19]:
model.eval()

Model(
  (first_linear): Linear(in_features=13, out_features=24, bias=True)
  (second_linear): Linear(in_features=24, out_features=32, bias=True)
  (third_linear): Linear(in_features=32, out_features=32, bias=True)
  (fourth_linear): Linear(in_features=32, out_features=8, bias=True)
  (fifth_linear): Linear(in_features=8, out_features=1, bias=True)
  (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [20]:
batch_size = 1

In [21]:
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "heart_attack.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})


In [22]:
onnx_model = onnx.load("/home/akanksha/code/heart_attack.onnx")
onnx.checker.check_model(onnx_model)

In [25]:
ort_session = onnxruntime.InferenceSession("/home/akanksha/code/heart_attack.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()